# Retrieval Dataset Analysis Report

## 1. Methodology: Causal Relevance Verification

This report analyzes the empirical relevance of documentation for solving software engineering tasks. Instead of relying on static similarity (embeddings) or human labeling, we use **Monte Carlo Causal Inference**. 

### The Algorithm
For each query $q$ and a pool of candidate documents $C(q)$: 
1.  **Randomized Trials:** We execute $N$ trials. In each trial, a random subset $S \subset C(q)$ is injected into the LLM's context window.
2.  **Blind Solving:** The LLM attempts to solve the task using *only* $S$.
3.  **Impact Scoring (Delta P):** We calculate the marginal contribution of each document $d$: 
    $$ \Delta P(d) = P(	ext{Success} | d \in S) - P(	ext{Success} | d 
otin S) $$ 
### Interpretation
*   **$\Delta P pprox 1.0$**: **Critical**. The task cannot be solved without this document.
*   **$\Delta P > 0.1$**: **Helpful**. The document improves success rate but might be redundant with others.
*   **$\Delta P pprox 0.0$**: **Irrelevant**. The document is noise.
*   **$\Delta P < -0.1$**: **Toxic**. The document actively misleads the model (e.g., hallucination trigger).

In [ ]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import os
import json
from pathlib import Path
import sys
# Add root to path to import lib
sys.path.append('..')
from tools.retrieval_dataset_generation.lib import RetrievalDataset, RetrievalCase

print(f"Current Working Directory: {os.getcwd()}")
# Configure plotting
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
pd.set_option('display.max_colwidth', 100)

In [ ]:
# Load Latest Run Logs (YAML Stream)
trial_events = []
convergence_events = []
trials_df = pd.DataFrame()

log_files = glob.glob("../logs/*.yaml")
if not log_files:
    # Try relative to root if running from there
    log_files = glob.glob("logs/*.yaml")

if not log_files:
    print("No log files found.")
else:
    latest_log = max(log_files, key=os.path.getctime)
    print(f"Loading logs from: {latest_log}")
    
    all_events = []
    with open(latest_log, 'r') as f:
        events = list(yaml.safe_load_all(f))
        all_events = events
    
    # Extract Trial Events
    trial_events = [e for e in all_events if e and e.get('event') == 'trial_complete']
    trials_df = pd.DataFrame(trial_events)
    
    # Extract Convergence Events
    convergence_events = [e for e in all_events if e and e.get('event') == 'convergence_check']
    
    print(f"Loaded {len(trials_df)} trials and {len(convergence_events)} convergence checks.")

## 2. Impact Score Distribution (Signal vs. Noise)

This Kernel Density Estimate (KDE) plot visualizes the distribution of utility across different candidate sources.

In [ ]:
# Load Dataset Metadata
dataset_path = Path("../retrieval_dataset_verified.yaml")
if not dataset_path.exists():
    dataset_path = Path("retrieval_dataset_verified.yaml")

if not dataset_path.exists():
    raise FileNotFoundError("Dataset file not found.")

print(f"Loading metadata from: {dataset_path}")
with open(dataset_path, 'r') as f:
    dataset = RetrievalDataset.model_validate(yaml.safe_load(f))

print(f"Loaded {len(dataset.cases)} cases.")

cases = dataset.cases
records = []
convergence_traces = []
for case in cases:
    # Use Pydantic dot notation or dict access on metadata dict
    if 'convergence_trace' in case.metadata:
        convergence_traces.append({
            'case_id': case.id,
            'trace': case.metadata['convergence_trace']
        })
    candidates = case.candidates
    for ctx in candidates:
        meta = ctx.metadata
        records.append({
            'case_id': case.id,
            'fqn': ctx.fqn,
            'source_type': ctx.context_type,
            'delta_p': meta.delta_p,
            'n_in': meta.n_in,
        })
stats_df = pd.DataFrame(records)

if not stats_df.empty:
    plt.figure(figsize=(12, 6))
    sns.kdeplot(data=stats_df, x='delta_p', hue='source_type', fill=True, common_norm=False)
    plt.title('Impact Score Density by Candidate Source')
    plt.xlabel('Delta P (Impact Score)')
    plt.axvline(0, color='red', linestyle='--')
    plt.show()

## 3. Case-Level Convergence (Max Uncertainty)

This plot shows the **Maximum Standard Error** across all candidates for each case. This metric determines when we stop testing a case.

In [ ]:
plt.figure(figsize=(12, 6))
for trace in convergence_traces:
    y = trace['trace']
    x = range(1, len(y) + 1)
    plt.plot(x, y, alpha=0.5, label=trace['case_id'][:20])

    plt.title('Convergence of Case Uncertainty (Max SE) over Trials')
    plt.xlabel('Trial Number')
    plt.ylabel('Max Standard Error')
    plt.axhline(0.1, color='red', linestyle='--', label='Threshold (0.1)')
    plt.tight_layout()
    plt.show()

## 4. Deep Dive: Per-Context Convergence

This visualization drills down into a single case, showing the uncertainty (Standard Error) for **each individual document** over time. 

**Interpretation:**
*   **Fan-out:** Lines start high (SE=1.0) and drop.
*   **Separation:** Some docs converge quickly (easy to judge), others stay high (noisy/ambiguous).
*   **The Governor:** The slowest-converging line (highest Y) is what holds up the entire case.

In [ ]:
# Select the first case found in logs
if convergence_events:
    example_case_id = convergence_events[0]['case_id']
    print(f"Analyzing specific convergence for case: {example_case_id}")
    
    # Extract SE history for this case
    se_history = []
    for e in convergence_events:
        if e['case_id'] == example_case_id:
            # Strict check: se_map is guaranteed by Pydantic model
            if 'se_map' in e:
                for fqn, se in e['se_map'].items():
                    se_history.append({
                        'trial': e['trial_index'],
                        'fqn': fqn,
                        'standard_error': se
                    })
    
    se_df = pd.DataFrame(se_history)
    
    if not se_df.empty:
        plt.figure(figsize=(14, 8))
        sns.lineplot(data=se_df, x='trial', y='standard_error', hue='fqn', legend=False, alpha=0.6, linewidth=1)
        
        # Highlight the max envelope
        max_se_per_trial = se_df.groupby('trial')['standard_error'].max()
        plt.plot(max_se_per_trial.index, max_se_per_trial.values, color='black', linestyle='--', linewidth=2, label='Max Envelope')
        
        plt.title(f'Per-Context Convergence Trajectories for {example_case_id[:30]}...')
        plt.ylabel('Standard Error (Uncertainty)')
        plt.xlabel('Trial Number')
        plt.legend()
        plt.show()

## 5. Trial Dynamics (Success Rate)

This graph tracks the stability of the model's success rate over the course of the Monte Carlo trials.

*   **Rising Curve:** The model success rate is stable or improving as we sample more contexts.
*   **High Variance (Jagged):** The model is **brittle**. Its success depends heavily on exactly which subset of documents is present in the context. This indicates that the "Winning Coalition" is rare or fragile.

In [ ]:
if not trials_df.empty:
    trials_df['success'] = trials_df['is_correct'].astype(int)
    trials_df['cumulative_success_rate'] = trials_df.groupby('case_id')['success'].expanding().mean().reset_index(level=0, drop=True)
    
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=trials_df, x='trial_index', y='cumulative_success_rate', hue='case_id', alpha=0.7)
    plt.title('Cumulative Success Rate by Case')
    plt.xlabel('Trial Index')
    plt.ylabel('Success Rate')
    plt.show()

## 6. Case Breakdown: Relevance Analysis

This section provides a detailed breakdown of candidate documents for **every case**, sorted by their causal relevance (Delta P).

**Legend:**
*   **$\Delta P > 0.1$:** **Primary Signal**. These documents are likely essential.
*   **Source Type:** Where this candidate came from (Gold Mining, Vector Search, or Random Noise).

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

if not stats_df.empty:
    # Group by Case ID
    cases_groups = stats_df.groupby('case_id')
    
    for case_id, group in cases_groups:
        print()
        print(f"# Case: {case_id}")
        
        # Get query text from dataset object (not in stats_df)
        case_obj = next((c for c in dataset.cases if c.id == case_id), None)
        if case_obj:
             print(f"**Query:** {case_obj.query}")
             print(f"**Zero-Context Success:** {case_obj.metadata.get('zero_context_success_rate', 'N/A')}")
        
        # Sort by Delta P descending
        sorted_group = group.sort_values('delta_p', ascending=False)
        
        # Categorize
        relevant = sorted_group[sorted_group['delta_p'] > 0.1]
        neutral = sorted_group[(sorted_group['delta_p'] <= 0.1) & (sorted_group['delta_p'] >= -0.1)]
        toxic = sorted_group[sorted_group['delta_p'] < -0.1]
        
        display_cols = ['fqn', 'delta_p', 'source_type', 'n_in']
        
        if not relevant.empty:
            print()
            print("### ✅ Relevant Documents (Delta P > 0.1)")
            print(relevant[display_cols].to_markdown(index=False))
        else:
            print()
            print("*No highly relevant documents found.*")
            
        if not toxic.empty:
             print()
             print("### ❌ Toxic Documents (Delta P < -0.1)")
             print(toxic[display_cols].to_markdown(index=False))